# Stage 03 — SQL Analysis (SQLite)

This stage uses **SQL exploration** to analyze the Falcon 9 dataset stored in a relational database.  
The goal is to extract insights on launch sites, payloads, booster versions, and landing outcomes, building the foundation for predictive modeling of first-stage recovery.

---


## Introduction

In this stage we:  
1. Load the Falcon 9 dataset into a SQLite database.  
2. Run structured SQL queries to answer engineering and business questions.  
3. Interpret results to support predictive modeling of Falcon 9 landings.  

SQL allows efficient data exploration, integrity checks, and aggregation — skills expected in real-world data science pipelines.  

---

## Dataset Overview  

- **Provider**: SpaceX (Falcon 9 launch history)  
- **Focus**: Payload, booster, launch site, and landing outcome records  
- **Relevance**: First-stage recovery determines whether launch costs remain at ~62M (SpaceX) or balloon toward ~165M (competitors).  

Being able to **predict landing success** translates directly into competitive bidding insights and operational forecasting.  

---

## Connecting to the Database  

The dataset is stored locally as a CSV and imported into SQLite.  
From here, SQL queries provide structured exploration.  

---

In [ ]:
!pip install sqlalchemy==1.4.1

In [ ]:
!pip install ipython-sql
!pip install ipython-sql prettytable

In [ ]:
%load_ext sql

In [ ]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [ ]:
!pip install -q pandas

In [ ]:
%sql sqlite:///my_data1.db

In [ ]:
import pandas as pd
df = pd.read_csv("Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

**Note:This below code is added to remove blank rows from table**


In [ ]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

In [ ]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

## SQL Exploration Tasks  

Each task is framed as a business/engineering question, followed by interpretation of the results.  


### Task 1 — Identify Unique Launch Sites  
**Why:** Establish operational diversity.  
**Insight:** Reveals the number of active SpaceX facilities.  

---

In [ ]:
%%sql
SELECT DISTINCT "Launch_Site"
FROM SPACEXTABLE;

### Task 2 — Launch Sites Beginning with "CCA"  
**Why:** Focus on Cape Canaveral launches.  
**Insight:** Validate data quality and confirm expected site codes.  

---

In [ ]:
%%sql
SELECT *
FROM SPACEXTABLE
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;

### Task 3 — Total Payload Mass for NASA (CRS) Missions  
**Why:** Quantify total cargo delivered under NASA’s CRS program.  
**Insight:** Demonstrates SpaceX’s logistics role with the ISS.  

---

In [ ]:
%%sql
SELECT SUM(PAYLOAD_MASS__KG_)
FROM SPACEXTABLE
Where Customer = "NASA (CRS)";

### Task 4 — Average Payload for Booster *F9 v1.1*  
**Why:** Benchmark payload capacity of an early Falcon 9 booster.  
**Insight:** Reference point for comparing later upgrades.  

---

In [ ]:
%%sql
SELECT AVG(PAYLOAD_MASS__KG_)
FROM SPACEXTABLE 
WHERE Booster_Version = "F9 v1.1";

### Task 5 — First Successful Ground Pad Landing  
**Why:** Locate milestone for reusability.  
**Insight:** Marks the date of the first recovery success on land.  

---

In [ ]:
%%sql
SELECT MIN(Date) 
FROM SPACEXTABLE
WHERE Landing_Outcome = "Success (ground pad)";

### Task 6 — Drone Ship Successes with Payloads 4,000–6,000 kg  
**Why:** Examine recovery reliability under medium-weight payloads.  
**Insight:** Helps correlate payload mass with landing success.  

---

In [ ]:
%%sql
SELECT Booster_Version 
FROM SPACEXTABLE
WHERE Landing_Outcome = "Success (drone ship)"
AND "Payload_Mass__kg_" > 4000
AND "Payload_Mass__kg_" < 6000;

### Task 7 — Count of Mission Outcomes  
**Why:** Assess overall reliability by tallying success vs. failure.  
**Insight:** High success ratio validates reusability model.  

---

In [ ]:
%%sql
SELECT
  SUM(CASE WHEN "Mission_Outcome" LIKE '%Success%' THEN 1 ELSE 0 END) AS total_success,
  SUM(CASE WHEN "Mission_Outcome" LIKE '%Failure%' THEN 1 ELSE 0 END) AS total_failure
FROM SPACEXTABLE;

### Task 8 — Identify the Booster with Maximum Payload  
**Why:** Highlight record-setting missions by payload mass.   
**Insight:** Demonstrates Falcon 9’s upper limits of payload delivery, a key engineering benchmark.  

---

In [ ]:
%%sql
SELECT "Booster_Version", "Payload_Mass__kg_"
FROM SPACEXTABLE
WHERE "Payload_Mass__kg_" = (
    SELECT MAX("Payload_Mass__kg_")
    FROM SPACEXTABLE
);

### Task 9 — Drone Ship Failures in 2015  
**Why:** Examine early challenges with offshore recovery attempts.  
**Insight:** This query highlights failed drone ship landings during 2015, capturing which boosters were involved and when the failures occurred.  

---

In [ ]:
%%sql
SELECT 
  SUBSTR("Date", 6, 2) AS month,
  "Landing_Outcome",
  "Booster_Version",
  "Launch_Site"
FROM SPACEXTABLE
WHERE 
  "Landing_Outcome" LIKE '%Failure%' 
  AND "Landing_Outcome" LIKE '%drone ship%'
  AND SUBSTR("Date", 1, 4) = '2015';


### Task 10 — Distribution of Landing Outcomes (2010–2017)  
**Why:** Summarize recovery performance during the critical early years of Falcon 9 launches.  
**Insight:** Reveals how often each landing outcome occurred in the early years of Falcon 9 development. This forms the baseline for evaluating SpaceX’s technological progress in rocket reusability.

---

In [ ]:
%%sql
SELECT 
  "Landing_Outcome",
  COUNT(*) AS outcome_count
FROM SPACEXTABLE
WHERE 
  "Date" BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY outcome_count DESC;


## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
